In [1]:
!pip install torch transformers sentencepiece datasets sacrebleu accelerate pyvi


Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 97.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 18.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 208.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 241.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 131.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 232.4 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python3 -m pip

In [2]:
# Cell 2: Setup and imports

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
from datasets import Dataset, concatenate_datasets
from transformers import (
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback,
    AutoTokenizer
)
import random

from pyvi import ViTokenizer

print(torch.cuda.get_device_name(0))
print("VRAM:", torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")

NVIDIA H200
VRAM: 150.0217344 GB


In [3]:
# Cell 3: Load Khmer tokenizer
# Load Khmer tokenizer from Hugging Face
khmer_word_tokenizer = AutoTokenizer.from_pretrained(
    "khopilot/km-tokenizer-khmer", 
    use_fast=False
)
print("Khmer tokenizer loaded successfully!")

# Cell 3b: Define tokenization functions
def tokenize_vietnamese(text):
    """Tokenize Vietnamese text using PyVi"""
    try:
        return ViTokenizer.tokenize(text)
    except Exception as e:
        print(f"Error tokenizing Vietnamese: {e}")
        return text

def tokenize_khmer(text):
    """Tokenize Khmer text using khopilot/km-tokenizer-khmer"""
    try:
        tokens = khmer_word_tokenizer.tokenize(text)
        return " ".join(tokens)
    except Exception as e:
        print(f"Error tokenizing Khmer: {e}")
        return text

def tokenize_batch_vietnamese(texts):
    """Batch tokenize Vietnamese texts"""
    print(f"Tokenizing {len(texts)} Vietnamese texts...")
    results = []
    for i, text in enumerate(texts):
        results.append(tokenize_vietnamese(text))
        if (i + 1) % 10000 == 0:
            print(f"  Processed {i + 1}/{len(texts)} Vietnamese texts")
    return results

def tokenize_batch_khmer(texts):
    """Batch tokenize Khmer texts"""
    print(f"Tokenizing {len(texts)} Khmer texts...")
    results = []
    for i, text in enumerate(texts):
        results.append(tokenize_khmer(text))
        if (i + 1) % 10000 == 0:
            print(f"  Processed {i + 1}/{len(texts)} Khmer texts")
    return results

print("Vietnamese and Khmer tokenizers loaded successfully!")

tokenizer_config.json:   0%|          | 0.00/196 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/164k [00:00<?, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Khmer tokenizer loaded successfully!
Vietnamese and Khmer tokenizers loaded successfully!


In [5]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(
    "vi_to_km/phase1/best"
).cuda()

tokenizer = AutoTokenizer.from_pretrained(
    "vi_to_km/phase1/best"
)

print("Phase 1 model loaded!")

Phase 1 model loaded!


In [6]:

for param in model.model.encoder.parameters():
    param.requires_grad = False

# Verify
total, frozen, trainable = 0, 0, 0
for name, param in model.named_parameters():
    n = param.numel()
    total += n
    if not param.requires_grad:
        frozen += n
    else:
        trainable += n

print(f"Total params    : {total/1e6:.1f}M")
print(f"Frozen params   : {frozen/1e6:.1f}M")
print(f"Trainable params: {trainable/1e6:.1f}M")

Total params    : 483.9M
Frozen params   : 282.3M
Trainable params: 201.6M


In [7]:
# Cell 6: Load and prepare data
DATA_DIR = "dataset"

def load_parallel(src_file, tgt_file):
    with open(src_file, encoding="utf-8") as f:
        src = [l.strip() for l in f]
    with open(tgt_file, encoding="utf-8") as f:
        tgt = [l.strip() for l in f]
    
    assert len(src) == len(tgt)

    print("Tokenizing Vietnamese texts...")
    src_tokenized = tokenize_batch_vietnamese(src)
    
    print("Tokenizing Khmer texts...")
    tgt_tokenized = tokenize_batch_khmer(tgt) 
    
    return Dataset.from_dict({
        "src_text": src_tokenized,
        "tgt_text": tgt_tokenized
    })



# Load data from train.khm and train.vi
full_dataset = load_parallel(
    f"{DATA_DIR}/train_khm.vi",
    f"{DATA_DIR}/train_khm.khm"
)

print(f"Total dataset size: {len(full_dataset)} examples")

# Split dataset
test_size = 1000
dev_size = 3000

test_start_idx = len(full_dataset) - test_size
dev_start_idx = test_start_idx - dev_size

test_dataset = full_dataset.select(range(test_start_idx, len(full_dataset)))
dev_dataset = full_dataset.select(range(dev_start_idx, test_start_idx))
train_dataset = full_dataset.select(range(0, dev_start_idx))

train_dataset = train_dataset.shuffle(seed=42)

print(f"Train dataset: {len(train_dataset)} examples (for training)")
print(f"Dev dataset  : {len(dev_dataset)} examples (for validation during training)")
print(f"Test dataset : {len(test_dataset)} examples (for final evaluation)")
print("\nData split and shuffle completed.")

Tokenizing Vietnamese texts...
Tokenizing 599999 Vietnamese texts...
  Processed 10000/599999 Vietnamese texts
  Processed 20000/599999 Vietnamese texts
  Processed 30000/599999 Vietnamese texts
  Processed 40000/599999 Vietnamese texts
  Processed 50000/599999 Vietnamese texts
  Processed 60000/599999 Vietnamese texts
  Processed 70000/599999 Vietnamese texts
  Processed 80000/599999 Vietnamese texts
  Processed 90000/599999 Vietnamese texts
  Processed 100000/599999 Vietnamese texts
  Processed 110000/599999 Vietnamese texts
  Processed 120000/599999 Vietnamese texts
  Processed 130000/599999 Vietnamese texts
  Processed 140000/599999 Vietnamese texts
  Processed 150000/599999 Vietnamese texts
  Processed 160000/599999 Vietnamese texts
  Processed 170000/599999 Vietnamese texts
  Processed 180000/599999 Vietnamese texts
  Processed 190000/599999 Vietnamese texts
  Processed 200000/599999 Vietnamese texts
  Processed 210000/599999 Vietnamese texts
  Processed 220000/599999 Vietnamese 

In [8]:
# Cell 7: Preprocessing function
MAX_LEN = 256

def preprocess(batch):
    tokenizer.src_lang = "vi"  # Changed from "lo" to "km" for Khmer
    tokenizer.tgt_lang = "km"

    inputs = tokenizer(
        batch["src_text"],
        truncation=True,
        max_length=MAX_LEN
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            batch["tgt_text"],
            truncation=True,
            max_length=MAX_LEN
        )

    inputs["labels"] = labels["input_ids"]
    return inputs

In [9]:
# Cell 8: Apply preprocessing
train_dataset = train_dataset.map(
    preprocess,
    batched=True,
    remove_columns=train_dataset.column_names,
    num_proc=8
)

dev_dataset = dev_dataset.map(
    preprocess,
    batched=True,
    remove_columns=dev_dataset.column_names,
    num_proc=8
)

Map (num_proc=8):   0%|          | 0/595999 [00:00<?, ? examples/s]

/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers

Map (num_proc=8):   0%|          | 0/3000 [00:00<?, ? examples/s]

/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/admin/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers

In [10]:
# Cell 9: Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    padding=True
)

In [16]:
# Cell 10: Training arguments
import torch._dynamo
torch._dynamo.config.suppress_errors = True

training_args_phase2 = TrainingArguments(
    output_dir="./vi_to_km/phase2",
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    lr_scheduler_type="linear",
    warmup_ratio=0.05,
    weight_decay=0.005,
    max_grad_norm=1.0,
    num_train_epochs=1,
    bf16=True,
    fp16=False,
    group_by_length=True,
    dataloader_num_workers=8,
    dataloader_pin_memory=True,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    report_to="none",
)


In [17]:
# Cell 11: Create trainer
trainer_phase2 = Trainer(
    model=model,
    args=training_args_phase2,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[
        EarlyStoppingCallback(early_stopping_patience=3)
    ]
)

/tmp/ipykernel_3827/3872925220.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_phase2 = Trainer(


In [18]:
# Cell 12: Train
print("\n" + "="*60)
print("PHASE 1: Training Vietnamese - Khmer model")
print("="*60 + "\n")

trainer_phase2.train()


PHASE 1: Training Vietnamese - Khmer model



Step,Training Loss,Validation Loss
500,0.896900,0.857996
1000,0.873300,0.835001
1500,0.853500,0.812967
2000,0.860800,0.798562


There were missing keys in the checkpoint model loaded: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=2329, training_loss=0.8810450012154618, metrics={'train_runtime': 494.728, 'train_samples_per_second': 1204.7, 'train_steps_per_second': 4.708, 'total_flos': 7.596092301798605e+16, 'train_loss': 0.8810450012154618, 'epoch': 1.0})

In [20]:
# Cell 13: Save model
trainer_phase2.save_model("./vi_to_km/phase2/best")
tokenizer.save_pretrained("./vi_to_km/phase2/best")
print("\nPhase 2 model saved!")


Phase 2 model saved!


In [21]:
# Cell 14: Evaluation functions
def translate_batch(texts, model, tokenizer, batch_size=32):
    """Batch translation for speed"""
    model.eval()
    tokenizer.src_lang = "vi"  # Changed from "lo" to "km"
    tokenizer.tgt_lang = "km"
    
    outputs = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        
        inputs = tokenizer(
            batch,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=256
        ).to("cuda")
        
        with torch.no_grad():
            gen = model.generate(
                **inputs,
                forced_bos_token_id=tokenizer.get_lang_id("km"),
                num_beams=5,
                max_length=256
            )
        
        texts_out = tokenizer.batch_decode(gen, skip_special_tokens=True)
        outputs.extend(texts_out)
        
        if (i // batch_size + 1) % 10 == 0:
            print(f"Translated {i+len(batch)}/{len(texts)}")
    
    return outputs

# Get test data
test_khm = test_dataset["tgt_text"]
test_vi = test_dataset["src_text"]

print(f"\nTest set size: {len(test_khm)} examples")
print("\nTranslating test set...")
preds_phase2 = translate_batch(test_vi, model, tokenizer)


Test set size: 1000 examples

Translating test set...
Translated 320/1000
Translated 640/1000
Translated 960/1000


In [22]:
# Cell 15: Calculate BLEU score
from sacrebleu import corpus_bleu

bleu_phase2 = corpus_bleu(preds_phase2, [test_khm])
print(f"\n{'='*60}")
print(f"PHASE 1 BLEU Score: {bleu_phase2.score:.2f}")
print(f"{'='*60}\n")

# Save predictions
with open("./vi_to_km/phase2/phase2_predictions_vi_km.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(preds_phase2))


PHASE 1 BLEU Score: 19.67



In [23]:
# Cell 16: Show sample translations
print("\n" + "="*70)
print("SAMPLE TRANSLATIONS")
print("="*70)

for i in range(10):
    print(f"\nExample {i+1}:")
    print(f"Source (Vietnamese): {test_vi[i]}")
    print(f"Reference (Khmer): {test_khm[i]}")
    print(f"Prediction: {preds_phase2[i]}")
    print("-" * 70)


SAMPLE TRANSLATIONS

Example 1:
Source (Vietnamese): Ủy_ban bầu_cử Myanmar đã bác_bỏ tuyên_bố này .
Reference (Khmer): ▁ គណៈកម្មការ រៀបចំការបោះឆ្នោត របស់ប្រទេស មីយ៉ាន់ម៉ា បាន ច្រ ា ន ចោល ការ អះអាង នេះ ។
Prediction: នេះបើតាមការ បញ្ជាក់ របស់ គណៈកម្មការ រៀបចំការបោះឆ្នោត មីយ៉ាន់ម៉ា ។
----------------------------------------------------------------------

Example 2:
Source (Vietnamese): Bộ_trưởng nói : " Phụ_nữ cũng muốn được lên_tiếng , được nêu lên những vấn_đề của chính mình . Trong quá_khứ , phụ_nữ nhận thấy rằng những người_làm chính_trị không thực_sự nêu lên những vấn_đề của chính họ vì bản_thân họ chưa từng đối_mặt với những vấn_đề đó . "
Reference (Khmer): ▁លោកស្រី រដ្ឋមន្ត្រី មានប្រសាសន៍ថា ៖ « ស្ត្រី ក៏ ចង់បាន ស្រ្តី ជា សំឡេង ខ្លួន ដែរ ▁ លើក បញ្ហា របស់ ខ្លួនឯង ▁ដែល កន្លងមក ស្ត្រី ឃើញថា ▁ អ្នកដែល ធ្វើ នយោបាយ ▁លោក មិនបាន លើក បញ្ហា ខ្លួន ឲ្យ ពិត ជាក់ស្តែង ▁ដោយសារ លោក មិនបាន ជួប បញ្ហា ហ្នឹង ដោយ ខ្លួន គាត់ ។
Prediction: លោក រដ្ឋមន្រ្តី ថ្លែងថា ៖ « ស្ត្រី ក៏ ចង់ឲ្យ មានការ និយាយ ដោយ លើកឡ